In [ ]:
import sys
import os
import warnings
from pathlib import Path
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
import time
import logging

# ---------------------------------------------------------------------------
# 1) SETUP LOGGING & ENVIRONMENT
# ---------------------------------------------------------------------------
warnings.filterwarnings("ignore")

# Logging setup
logging.basicConfig(
    filename="models/saved_models/pair_trading.log",
    level=logging.INFO,
    format="%(asctime)s %(levelname)s:%(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

def log_and_print(message, is_error=False):
    if is_error:
        logging.error(message)
    else:
        logging.info(message)
    print(message)

# ---------------------------------------------------------------------------
# 2) ACCOUNT CONFIGURATION
# ---------------------------------------------------------------------------
name = 7889999
key = "hdgdggFxEG38"
serv = "ICMarketsSC-Demo"

# Global parameters
PAIR1 = "AUDUSD"
PAIR2 = "NZDUSD"
LOT_SIZE = 0.01
TIMEFRAME = mt5.TIMEFRAME_H1  # Adjust as needed
N_BARS = 5000  # More data for stable pair trading
MAGIC_NUMBER = 77777
SLEEP_TIME = 3600  # 1 Hour sleep cycle

# ---------------------------------------------------------------------------
# 3) MT5 FUNCTIONS
# ---------------------------------------------------------------------------
def get_data(symbol, n, timeframe):
    """Fetches historical price data from MT5."""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None:
        raise ValueError(f"Could not retrieve data for {symbol}")
    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df[["close"]]

def check_cointegration(symbol1, symbol2, n_bars):
    """Performs cointegration test between two assets."""
    df1 = get_data(symbol1, n_bars, TIMEFRAME)
    df2 = get_data(symbol2, n_bars, TIMEFRAME)

    score, p_value, _ = coint(df1["close"], df2["close"])
    log_and_print(f"⚖️ Cointegration Test p-value ({symbol1} & {symbol2}): {p_value:.4f}")
    
    return p_value < 0.05  # True if cointegrated

def compute_z_score(spread, lookback=60):
    """Calculates the rolling Z-score of the spread."""
    mean = spread.rolling(lookback).mean()
    std = spread.rolling(lookback).std()
    return (spread - mean) / std

def generate_pair_signals(pair1, pair2, lookback=60, entry_threshold=1.5, exit_threshold=0.5):
    """Computes Z-score signals for pair trading."""
    df1 = get_data(pair1, N_BARS, TIMEFRAME)
    df2 = get_data(pair2, N_BARS, TIMEFRAME)

    # Compute spread
    spread = df1["close"] - df2["close"]
    z_score = compute_z_score(spread, lookback)

    df = pd.DataFrame({"spread": spread, "z_score": z_score})
    
    # Trading signals
    df["long_signal"] = df["z_score"] < -entry_threshold  # Buy Pair1, Sell Pair2
    df["short_signal"] = df["z_score"] > entry_threshold  # Sell Pair1, Buy Pair2
    df["exit_signal"] = df["z_score"].abs() < exit_threshold  # Exit trade

    return df

def place_order(symbol, lot, is_buy, magic):
    """Places an order on MT5."""
    order_type = mt5.ORDER_TYPE_BUY if is_buy else mt5.ORDER_TYPE_SELL
    deviation = 20

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "deviation": deviation,
        "magic": magic,
        "comment": "Pair Trading Bot",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result is None or result.retcode != mt5.TRADE_RETCODE_DONE:
        log_and_print(f"❌ Order failed for {symbol}: {result.retcode}", is_error=True)
    else:
        log_and_print(f"✅ Order placed for {symbol}")

def manage_trades(pair1, pair2, lot_size, signals):
    """Executes pair trading strategy based on signals."""
    log_and_print(f"📈 Running Pair Trading Strategy for {pair1} & {pair2}")

    latest_signal = signals.iloc[-1]

    if latest_signal["long_signal"]:
        log_and_print(f"📉 Enter SHORT {pair2} & LONG {pair1}")
        place_order(pair1, lot_size, is_buy=True, magic=MAGIC_NUMBER)
        place_order(pair2, lot_size, is_buy=False, magic=MAGIC_NUMBER)

    elif latest_signal["short_signal"]:
        log_and_print(f"📈 Enter LONG {pair2} & SHORT {pair1}")
        place_order(pair1, lot_size, is_buy=False, magic=MAGIC_NUMBER)
        place_order(pair2, lot_size, is_buy=True, magic=MAGIC_NUMBER)

    elif latest_signal["exit_signal"]:
        log_and_print(f"❌ Exiting positions for {pair1} & {pair2}")
        close_positions(pair1, MAGIC_NUMBER)
        close_positions(pair2, MAGIC_NUMBER)

def close_positions(symbol, magic):
    """Closes all open positions for a given symbol and magic number."""
    positions = mt5.positions_get(symbol=symbol)
    if positions:
        for pos in positions:
            if pos.magic == magic:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_BUY if pos.type == mt5.ORDER_TYPE_SELL else mt5.ORDER_TYPE_SELL,
                    "position": pos.ticket,
                    "magic": magic,
                    "comment": "Closing Pair Trade",
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    log_and_print(f"✅ Closed position for {symbol}")
                else:
                    log_and_print(f"❌ Failed to close position for {symbol}", is_error=True)

# ---------------------------------------------------------------------------
# 4) MAIN TRADING LOOP
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    try:
        if not mt5.initialize(login=name, server=serv, password=key):
            log_and_print("Failed to initialize MetaTrader 5", is_error=True)
            exit()

        # Validate Cointegration
        if not check_cointegration(PAIR1, PAIR2, N_BARS):
            log_and_print(f"⚠️ {PAIR1} & {PAIR2} are NOT cointegrated. Exiting.", is_error=True)
            mt5.shutdown()
            exit()

        while True:
            log_and_print("🔄 Fetching New Data & Running Strategy...")

            # Generate signals
            signals = generate_pair_signals(PAIR1, PAIR2)

            # Execute trades
            manage_trades(PAIR1, PAIR2, LOT_SIZE, signals)

            # Sleep before next check
            time.sleep(SLEEP_TIME)

    except KeyboardInterrupt:
        log_and_print("Shutdown signal received.")
    except Exception as e:
        log_and_print(f"An error occurred: {e}", is_error=True)
    finally:
        mt5.shutdown()
        log_and_print("MetaTrader 5 shutdown completed.")
